In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [2]:
engine = create_engine(
    "mysql+pymysql://root:@localhost:3306/accounting_system_myself"
)


In [3]:
stores = pd.read_sql("""
SELECT *
FROM stores st
JOIN store_translations stt 
    ON st.id = stt.store_id
WHERE st.deleted_at IS NULL
  AND stt.locale = 'ar'
""", engine)

In [6]:
stores[['name','manager_name','address','phone','status']]

,name,manager_name,address,phone,status
0,المخزن الرئيسي,أحمد علي,صنعاء - شارع حدة,777123456,1
1,المخزن الفرعي,محمد صالح,تعز - شارع الستين,777654321,1


In [7]:
categories = pd.read_sql("""
SELECT *
FROM categories ct
JOIN category_translations ctt 
    ON ct.id = ctt.category_id
WHERE ct.deleted_at IS NULL
  AND ctt.locale = 'ar'
""", engine)

In [8]:
categories[['name','status']]

,name,status
0,الأدوات المكتبية,1
1,الأجهزة الإلكترونية,1
2,المواد الاستهلاكية,1


In [9]:
units = pd.read_sql("""
SELECT *
FROM units ut
JOIN unit_translations utt 
    ON ut.id = utt.unit_id
WHERE ut.deleted_at IS NULL
  AND utt.locale = 'ar'
""", engine)

In [11]:
units[['name','conversion_factor']]

,name,conversion_factor
0,قطعة,1.0
1,كيلوجرام,1.0
2,لتر,1.0
3,متر,1.0


In [12]:
products = pd.read_sql("""
SELECT *
FROM products pt
JOIN product_translations ptt 
    ON pt.id = ptt.product_id
WHERE pt.deleted_at IS NULL
  AND ptt.locale = 'ar'
""", engine)

In [22]:
df = products.merge(
    units,
       left_on="unit_id",
    right_on="unit_id",
    how="left",
    suffixes=("", "_unit")
)

In [25]:
dfs = df.merge(
    categories,
       left_on="category_id",
    right_on="category_id",
    how="left",
    suffixes=("", "_category")
)

In [27]:
dfs[['name','name_category','name_unit','price','image','status']]

,name,name_category,name_unit,price,image,status
0,قلم,الأدوات المكتبية,قطعة,5000,pen.jpg,1
1,دفتر,الأدوات المكتبية,قطعة,20000,notebook.jpg,1
2,آلة حاسبة,الأجهزة الإلكترونية,قطعة,150000,calculator.jpg,1
3,تابلت,الأجهزة الإلكترونية,قطعة,500000,tablet.jpg,1
4,أرز كيلو,المواد الاستهلاكية,كيلوجرام,20000,rice_kg.jpg,1
5,حليب لتر,المواد الاستهلاكية,لتر,10000,milk_liter.jpg,1


In [29]:
inventory_df = pd.read_sql("""
SELECT *
FROM inventories
WHERE deleted_at IS NULL
""", engine)

In [36]:
products_df = pd.read_sql("""
SELECT *
FROM products
WHERE deleted_at IS NULL
""", engine)

In [43]:
units = pd.read_sql("""
SELECT *
FROM units ut
JOIN unit_translations utt 
    ON ut.id = utt.unit_id
WHERE ut.deleted_at IS NULL
  AND utt.locale = 'ar'
""", engine)

In [45]:
units

,id,conversion_factor,created_at,updated_at,deleted_at,id,unit_id,locale,name
0,1,1.0,2026-01-13 12:35:01,None,None,1,1,ar,قطعة
1,2,1.0,2026-01-13 12:35:01,None,None,3,2,ar,كيلوجرام
2,3,1.0,2026-01-13 12:35:01,None,None,5,3,ar,لتر
3,4,1.0,2026-01-13 12:35:01,None,None,7,4,ar,متر


In [60]:
inventory_summary = (
    inventory_df
    .groupby('product_id', as_index=False)
    .agg(
        total_in=('quantity_in', 'sum'),
        total_out=('quantity_out', 'sum')
    )
)

# 2️⃣ دمج المنتجات مع المخزون
products_inventory = products.merge(
    inventory_summary,
    left_on='product_id',
    right_on='product_id',
    how='left'
)

# 3️⃣ تعويض القيم الفارغة بـ 0
products_inventory[['total_in', 'total_out']] = (
    products_inventory[['total_in', 'total_out']]
    .fillna(0)
)

# 4️⃣ حساب الصافي (system_quantity)
products_inventory['system_quantity'] = (
    products_inventory['total_in'] - products_inventory['total_out']
)

# 5️⃣ دمج وحدة القياس
products_inventory = products_inventory.merge(
    units,
    left_on='unit_id',
    right_on='unit_id',
    how='left',
    suffixes=('', '_unit')
)
# 6️⃣ تجهيز النتيجة النهائية (مثل transform في Laravel)
final_df = products_inventory[['id', 'name', 'name_unit', 'system_quantity']].rename(
    columns={
        'id': 'product_id',
        'name_unit': 'unit_name'
    }
)

# actual_quantity = system_quantity
final_df['actual_quantity'] = final_df['system_quantity']


final_df[['name','unit_name','system_quantity','actual_quantity']]


,name,unit_name,system_quantity,actual_quantity
0,قلم,قطعة,100.0,100.0
1,دفتر,قطعة,100.0,100.0
2,آلة حاسبة,قطعة,10.0,10.0
3,تابلت,قطعة,0.0,0.0
4,أرز كيلو,كيلوجرام,0.0,0.0
5,حليب لتر,لتر,0.0,0.0
